In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import datetime 
import time 


In [2]:
pttUrl = "https://www.ptt.cc"
fundUrl = pttUrl +"/bbs/Fund"

In [3]:
r = requests.get(fundUrl) #將此頁面的HTML GET下來
print(r.text) #印出HTML

<!DOCTYPE html>
<html>
	<head>
		<meta charset="utf-8">
		

<meta name="viewport" content="width=device-width, initial-scale=1">

<title>看板 Fund 文章列表 - 批踢踢實業坊</title>

<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-common.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-base.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-custom.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/pushstream.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-print.css" media="print">




	</head>
    <body>
		
<div id="topbar-container">
	<div id="topbar" class="bbs-content">
		<a id="logo" href="/bbs/">批踢踢實業坊</a>
		<span>&rsaquo;</span>
		<a class="board" href="/bbs/Fund/index.html"><span class="board-label">看板 </span>Fund</a>
		<a class="right small" href="/about.html">關於我們</a>
		<a class="right small" href="/contact.html">聯絡資

## 抓出最後一頁

In [4]:
soup = BeautifulSoup(r.text,"lxml") #將網頁資料以lxml
s = soup.findAll("a") 
p = []
for s in s:
    href = str(s.get('href'))
    matchObj = re.search('index[0-9]{2,}\\.html',href)
    if  matchObj != None :
        p.append(href)


In [5]:
lastPageUrl = pttUrl + str(p[0])
lastPageUrl

'https://www.ptt.cc/bbs/Fund/index1076.html'

In [6]:
latestPage = int(re.findall('[0-9]+', str(p))[0])
latestPage

1076

## 從最後一頁開始回推 

In [7]:
linksArticle = []
pageLength = 1
for pageIndex in range((latestPage-pageLength),latestPage+1):
    link = str(fundUrl) + "/index" + str(pageIndex) + ".html"
    print(link)
    r = requests.get(link) #將網頁資料GET下來
    soup = BeautifulSoup(r.text,"lxml") #將網頁資料以lxml
    sel = soup.select("div.title a") #取HTML標中的 <div class="title"></div> 中的<a>標籤存入sel
    links = [sel.get("href") for sel in sel]
    links = list(links)
    linksArticle = linksArticle + links

https://www.ptt.cc/bbs/Fund/index1075.html
https://www.ptt.cc/bbs/Fund/index1076.html


In [8]:
len(linksArticle)

40

In [9]:
linksArticle = list(dict.fromkeys(linksArticle))#remove duplicate

In [10]:
len(linksArticle)

40

## 抓取文章與推文

In [11]:
articleTable = pd.DataFrame() # 建⽴⽂章 儲存空間
articlePushTable= pd.DataFrame() # 建⽴每篇文章推⽂儲存空間
pushTable = pd.DataFrame() # 建⽴推⽂儲存空間

In [12]:
for tempLink in linksArticle:
    articleUrl = pttUrl + tempLink # ⽂章網址
    r = requests.get(articleUrl) #將網頁資料GET下來
    soup = BeautifulSoup(r.text,"lxml") #將網頁資料以lxml
    
    #內文
    sel = soup.select("span.article-meta-value") #取內文
    author = sel[0].text.split(' ')[0]
    category = sel[1].text
    title = sel[2].text
    dateTime = sel[3].text
    content = soup.find(id="main-content").text
    target_content = u'※ 發信站: 批踢踢實業坊(ptt.cc),'
    #去除掉 target_content
    content = content.split(target_content)
    #去除掉 datetime
    content = content[0].split(dateTime)
    #去除掉文末 --
    main_content = content[1].replace('--', '').replace("\n","").replace("\\","\\\\").replace('"',r'\"')
    #調整datetime格式
#     struct_time = time.strptime(str(dateTime),"%a %b %d %H:%M:%S %Y") # 轉成時間元組
#     dateTime = time.strftime("%Y-%m-%d %H:%M:%S", struct_time)
    tmp = pd.DataFrame({
        'datetime' : [dateTime],
        'title' : [title],
        'author' : [author],
        'content' : [main_content],
        'url' : [articleUrl]
    })
    articleTable = articleTable.append(tmp)
    
    #推文
    for tag in soup.select('div.push'):
        # pushTag  推文標籤  推  噓  註解(→)
        pushTag = tag.find("span", {'class': 'push-tag'}).text
        # print "push_tag:",push_tag

        # pushUserid 推文使用者id
        pushUserid = tag.find("span", {'class': 'push-userid'}).text
        # print "push_userid:",push_userid

        # pushContent 推文內容
        pushContent = tag.find("span", {'class': 'push-content'}).text
        pushContent = pushContent[1:]
        # print "push_content:",push_content

        # push-ipdatetime 推文時間
        pushIpDatetime = tag.find("span", {'class': 'push-ipdatetime'}).text
        pushIpDatetime = pushIpDatetime.rstrip().replace('/','-')
        
        tmp = pd.DataFrame({
            'tag' : [pushTag],
            'userid' : [pushUserid],
            'content' : [pushContent],
            'datetime' : [pushIpDatetime],
            'url' : [articleUrl]
        })
        articlePushTable = articlePushTable.append(tmp)
    pushTable = pushTable.append(articlePushTable)

articleTable.reset_index(drop=True, inplace=True) # index reset
pushTable.reset_index(drop=True, inplace=True) # index reset

In [13]:
articleTable.head()

,datetime,title,author,content,url
0,2021-08-07 08:09:48,[新聞] 基金特蒐／越南基金 長線潛力足,charles0939,1.原文連結：https://money.udn.com/money/story/5618/...,https://www.ptt.cc/bbs/Fund/M.1628294990.A.2C6...
1,2021-08-10 13:18:07,[問題] 在銀行買的基金配息作業日約多久 ?,SophiaH,"我有一筆基金在8月初年度配息,當初透過銀行購買(年輕時 不瞭原來另有鉅亨、基富通這些省手續費...",https://www.ptt.cc/bbs/Fund/M.1628572689.A.5EF...
2,2021-08-10 20:17:03,[專欄] 生技類股旺一夏 今年投資湊熱鬧,charles0939,生技類股旺一夏 今年投資湊熱鬧2021/08/06 14:00創富礦工美股財報季開跑，生技類...,https://www.ptt.cc/bbs/Fund/M.1628597826.A.708...
3,2021-08-11 16:41:21,[新聞] 台股基金十強 今年平均漲54%,charles0939,1.原文連結：https://ctee.com.tw/news/fund/500742.ht...,https://www.ptt.cc/bbs/Fund/M.1628671283.A.7F9...
4,2021-08-11 21:32:17,[新聞] 基富通歡慶奧運送單筆申購手續費0元酷碰,hirono,1.原文連結：https://www.fundrich.com.tw/event/Campa...,https://www.ptt.cc/bbs/Fund/M.1628688747.A.7DB...


In [14]:
pushTable

,tag,userid,content,datetime,url
0,推,bluezero000,母湯,08-07 09:52,https://www.ptt.cc/bbs/Fund/M.1628294990.A.2C6...
1,推,SirMaoMao,中信越南基金從8/9起就要暫停美元級別的單筆申購了,08-07 11:42,https://www.ptt.cc/bbs/Fund/M.1628294990.A.2C6...
2,推,shakira918,還沒聽過基金長線沒有用的,08-07 13:40,https://www.ptt.cc/bbs/Fund/M.1628294990.A.2C6...
3,→,charles0939,剛出社會時，買過一檔基金，後來認賠出場，再過幾年,08-07 14:00,https://www.ptt.cc/bbs/Fund/M.1628294990.A.2C6...
4,→,charles0939,被「清算」了,08-07 14:00,https://www.ptt.cc/bbs/Fund/M.1628294990.A.2C6...
...,...,...,...,...,...
20342,推,peaceever,施羅德網頁配息日是09/07,09-06 11:52,https://www.ptt.cc/bbs/Fund/M.1630898182.A.BB5...
20343,→,wche095,謝謝你們,09-06 12:22,https://www.ptt.cc/bbs/Fund/M.1630898182.A.BB5...
20344,推,shakira918,建議用MoneyDJ查，有準確的除息日跟發放日，一般網站,09-08 06:22,https://www.ptt.cc/bbs/Fund/M.1630898182.A.BB5...
20345,→,shakira918,打的的是除息日,09-08 06:22,https://www.ptt.cc/bbs/Fund/M.1630898182.A.BB5...


In [15]:
linksArticle = list(dict.fromkeys(pushTable['url']))#remove duplicate
len(linksArticle)

40

In [17]:
articleTable.dropna().shape

(40, 5)

In [18]:
pushTable.dropna().shape

(20347, 5)

In [19]:
articleTable.to_csv("pttfund0909_utf8.csv", index=False, encoding='utf_8_sig')

In [20]:
csv_name="pttfund0909_utf8.csv"

In [21]:
data = pd.read_csv(csv_name)
data = data.dropna(subset=['content'])
data

,datetime,title,author,content,url
0,2021-08-07 08:09:48,[新聞] 基金特蒐／越南基金 長線潛力足,charles0939,1.原文連結：https://money.udn.com/money/story/5618/...,https://www.ptt.cc/bbs/Fund/M.1628294990.A.2C6...
1,2021-08-10 13:18:07,[問題] 在銀行買的基金配息作業日約多久 ?,SophiaH,"我有一筆基金在8月初年度配息,當初透過銀行購買(年輕時 不瞭原來另有鉅亨、基富通這些省手續費...",https://www.ptt.cc/bbs/Fund/M.1628572689.A.5EF...
2,2021-08-10 20:17:03,[專欄] 生技類股旺一夏 今年投資湊熱鬧,charles0939,生技類股旺一夏 今年投資湊熱鬧2021/08/06 14:00創富礦工美股財報季開跑，生技類...,https://www.ptt.cc/bbs/Fund/M.1628597826.A.708...
3,2021-08-11 16:41:21,[新聞] 台股基金十強 今年平均漲54%,charles0939,1.原文連結：https://ctee.com.tw/news/fund/500742.ht...,https://www.ptt.cc/bbs/Fund/M.1628671283.A.7F9...
4,2021-08-11 21:32:17,[新聞] 基富通歡慶奧運送單筆申購手續費0元酷碰,hirono,1.原文連結：https://www.fundrich.com.tw/event/Campa...,https://www.ptt.cc/bbs/Fund/M.1628688747.A.7DB...
5,2021-08-12 11:56:06,[問題] 疫情來襲，是否看好這些基金?,Inred004,小弟從2019年的第一波疫情來襲後，一直都有在注意科技類基金近期將一些報酬40%左右的基金贖...,https://www.ptt.cc/bbs/Fund/M.1628740568.A.DC0...
6,2021-08-12 21:02:59,[問題] 第一銀行基金轉基富通,SBTAL,目前在第一銀行每個月定期定額扣款聯博全高收覺得手續費很貴，長期累積也是不小金額但是已經要扣2...,https://www.ptt.cc/bbs/Fund/M.1628773381.A.034...
7,2021-08-13 11:08:57,[問題] NN(L)新興市場(南非幣),viying,"大家好~最近有親戚推薦我買新興市場的債券(南非幣)推薦理由: 債券型基金風險很低, 除非政府...",https://www.ptt.cc/bbs/Fund/M.1628824141.A.35E...
8,2021-08-13 11:11:14,Re: [新聞] 基富通歡慶奧運送單筆申購手續費0元酷碰,muzi146,這邊新手戶想請教，因為有看到回文酷碰券不能用於定期定額所以我平時定期定額的基金，像是安聯台灣...,https://www.ptt.cc/bbs/Fund/M.1628824276.A.EC3...
9,2021-08-13 16:09:04,Re: [問題] NN(L)新興市場(南非幣),pupuliao,※ 引述《viying (ying)》之銘言：: 大家好~最近有親戚推薦我買新興市場的債券(...,https://www.ptt.cc/bbs/Fund/M.1628842146.A.736...


In [ ]:
articleTable['timetime'] = pd.to